In [1]:
import sys
import os

In [5]:
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import toml
import torch
import torch.nn as nn

In [6]:
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from IPython.display import display, HTML
from itertools import combinations_with_replacement

In [7]:
sys.path.append(os.path.join(sys.path[0], '../..'))

import endure.data.io as EndureIO
import endure.lsm.cost_model as CostFunc

from endure.ltune.data.generator import LTuneGenerator
from endure.ltune.data.dataset import LTuneIterableDataSet
from endure.lcm.model.builder import LearnedCostModelBuilder

In [8]:
config = EndureIO.Reader.read_config('../../endure.toml')

In [4]:
class ClassicTuner(nn.Module):
    def __init__(self, config: dict[str, ...]):
        super().__init__()
        self.params = config['ltune']['model']['classic']
        size_ratio_range = (config['lsm']['size_ratio']['max']
                            - config['lsm']['size_ratio']['min']
                            + 1)

        modules = []
        num_feat = config['ltune']['model']['in_dim']
        out_dim = 1 + size_ratio_range
        modules.append(nn.Linear(num_feat, out_dim))
        nn.init.xavier_normal_(modules[-1].weight)
        modules.append(nn.LeakyReLU())
        for _ in range(self.params['hidden_layers']):
            modules.append(nn.Linear(out_dim, out_dim))
            nn.init.xavier_normal_(modules[-1].weight)
            modules.append(nn.LeakyReLU())

        self.bits = nn.Sequential(
            nn.Linear(out_dim, 1),
        )

        self.size_ratio = nn.Sequential(
            nn.Linear(out_dim, size_ratio_range),
        )
        self.sigmoid = nn.Sigmoid()

        self.layers = nn.Sequential(*modules)

    def forward(self, x, temp=0.1, hard=False) -> torch.Tensor:
        out = self.layers(x)
        size_ratio = self.size_ratio(out)
        size_ratio = nn.functional.gumbel_softmax(
            size_ratio,
            tau=temp,
            hard=hard
        )
        print(f't gumbel = {size_ratio}')
        h = self.bits(out)

        return torch.concat([h, size_ratio], dim=-1)

In [50]:
class LearnedCostModelLoss(torch.nn.Module):
    def __init__(self, config: dict[str, ...], model_path: str):
        super().__init__()
        self.lcm_builder = LearnedCostModelBuilder(config)
        self.model = self.lcm_builder.build_model()
        _, extension = os.path.splitext(model_path)
        is_checkpoint = (extension == '.checkpoint')
        data = torch.load(os.path.join(config['io']['data_dir'], model_path))
        if is_checkpoint:
            data = data['model_state_dict']
        status = self.model.load_state_dict(data)
        
        # self._mean = torch.Tensor(config['lcm']['data']['mean_bias'])
        # self._std = torch.Tensor(config['lcm']['data']['std_bias'])
        assert (len(status.missing_keys) == 0)
        assert (len(status.unexpected_keys) == 0)

    def forward(self, pred, label):
        # For learned cost model loss, pred is the DB configuration, label is
        # the workload
        bpe = ((pred[:, 0] - 5) / 2.88).view(-1, 1)
        print(f'bpe = {bpe}')
        size_ratio = torch.argmax(pred[:, 1:], dim=-1).view(-1, 1)
        print(f'size_ratio = {size_ratio}')
        inputs = torch.concat([bpe, label, size_ratio], dim=-1)

        return self.model(inputs).sum(dim=-1).mean()

In [41]:
torch.Tensor(config['lcm']['data']['mean_bias'])

tensor([5.0000, 0.5000, 0.5000, 0.5000, 0.5000])

In [42]:
train_dataset = LTuneIterableDataSet(config, os.path.join(config['io']['data_dir'], 'train-data/workload-parquet'), shuffle=True)
train_data = DataLoader(train_dataset, batch_size=2, drop_last=True, num_workers=1)

test_dataset = LTuneIterableDataSet(config, os.path.join(config['io']['data_dir'], 'test-data/workload-parquet'), shuffle=False)
test_data = DataLoader(train_dataset, batch_size=262114, drop_last=True, num_workers=1)

In [52]:
model = ClassicTuner(config)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = LearnedCostModelLoss(config, model_path='/data/models/level-02-25/best.model')

In [44]:
def train_step(label, features):
    optimizer.zero_grad()
    pred = model(features, temp=0.1, hard=False)
    loss = loss_fn(pred, label)
    loss.backward()
    optimizer.step()
    
    return loss

In [45]:
# model.train()
# pbar = tqdm(train_data)
# train_len = 0
# total_loss = 0
# for batch, (label, feature) in enumerate(pbar):
#     loss = train_step(label, feature)
#     total_loss += loss
#     if batch % (100) == 0:
#         pbar.set_description(f'loss {loss:e}')

# if train_len == 0:
#     train_len = batch + 1
    
# print(total_loss.item() / train_len)

In [54]:
label, feature = next(iter(train_data))

In [55]:
pred = model(feature)
pred

out = tensor([[0.0815, 0.0000, 0.0000, 0.3074, 0.4437, 0.9142, 0.0000, 0.5347, 0.5568,
         0.0000, 0.0000, 0.3688, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.4142,
         0.1886, 0.2064, 0.1307, 0.2196, 0.0505, 0.4013, 0.0000, 0.3229, 0.1442,
         0.4675, 0.2901, 0.0000, 0.0000, 0.4141, 0.0000, 0.2329, 0.2210, 0.0000,
         0.2363, 0.2924, 0.3720, 0.1871, 0.0983, 0.0000, 0.1649, 0.3223, 0.1346,
         0.0000, 0.1666, 0.6351, 0.3379, 0.0026],
        [0.0178, 0.0000, 0.0000, 0.4414, 0.0000, 0.1314, 0.0000, 0.2797, 0.4673,
         0.0000, 0.0000, 0.2658, 0.0000, 0.0224, 0.0000, 0.0000, 0.0000, 0.2770,
         0.1260, 0.1578, 0.1142, 0.1776, 0.0000, 0.1756, 0.0000, 0.1501, 0.2117,
         0.1449, 0.0000, 0.0000, 0.0000, 0.2147, 0.0000, 0.1281, 0.1078, 0.0430,
         0.0832, 0.4263, 0.1983, 0.1381, 0.2196, 0.0000, 0.0000, 0.1367, 0.0000,
         0.0000, 0.0000, 0.6233, 0.3801, 0.0000]], grad_fn=<ReluBackward0>)
size_ratio_intermediate = tensor([[0.0815, 0.0000, 0.0000,

tensor([[0.0000e+00, 4.1825e-19, 1.2494e-28, 9.4802e-25, 2.0578e-30, 2.1708e-25,
         1.9035e-20, 2.2102e-20, 5.8653e-23, 6.0511e-17, 9.8905e-17, 6.7584e-24,
         7.2134e-20, 6.0915e-26, 1.2744e-19, 3.7841e-19, 1.3679e-25, 1.3473e-22,
         7.2265e-20, 9.6448e-12, 1.7624e-21, 1.6841e-21, 6.3368e-22, 9.1649e-18,
         4.4328e-20, 3.9145e-26, 6.5912e-20, 7.7627e-15, 1.3967e-12, 2.3007e-11,
         1.1238e-09, 1.4202e-24, 3.6824e-11, 2.3211e-22, 5.5700e-20, 1.0000e+00,
         2.1354e-26, 1.0756e-26, 3.8920e-27, 4.9729e-13, 1.5327e-14, 1.3886e-23,
         1.8193e-18, 2.5425e-14, 2.5695e-25, 3.3905e-22, 3.7189e-20, 6.0544e-21,
         4.5069e-18, 8.9756e-24],
        [0.0000e+00, 1.3630e-14, 2.5190e-19, 6.7134e-25, 5.3612e-27, 6.7026e-12,
         3.8695e-24, 1.0343e-12, 8.2580e-22, 6.6054e-26, 2.5128e-24, 2.7464e-20,
         3.2281e-28, 1.7983e-18, 2.0894e-25, 3.8247e-18, 8.0432e-22, 4.8198e-19,
         1.5552e-27, 1.6750e-13, 3.7183e-28, 3.7944e-20, 1.6697e-30, 6.4991

In [56]:
loss_fn(pred, label)

bpe = tensor([[-1.7361],
        [-1.7361]], grad_fn=<ViewBackward0>)
size_ratio = tensor([[34],
        [24]])


tensor(0., grad_fn=<MeanBackward0>)